In [24]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
!pip install tqdm
from tqdm import tqdm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv("processed_train.csv")

In [4]:
#Removing these for this MLP model only
df = df.drop(columns=["Unnamed: 0", "TRIP_ID", "ORIGIN_CALL", "ORIGIN_STAND", "TAXI_ID", "TIMESTAMP", "YR"])

In [32]:
#Take 5% of data to train
df_sample = df.sample(frac=0.05)

In [33]:
df_len = df["LEN"]
df_sample = df_sample.drop(columns=["LEN"])

In [34]:
len(df_sample)

85533

In [35]:
def create_feature(datum):
#     features = [datum['CALL_TYPE'],
#                 #datum['ORIGIN_CALL'] if not pd.isna(datum['ORIGIN_CALL']) else 0,
#                 #datum['ORIGIN_STAND'] if not pd.isna(datum['ORIGIN_STAND']) else 0,
#                 #datum['TAXI_ID'],
#                 datum['DAY_TYPE'],
#                 datum['YR'],
#                 datum['MON'],
#                 datum['DAY'],
#                 datum['HR'],
#                 datum['WK']
#                ]
    
    return datnum.values

features = [df_sample.iloc[i].values for i in range(len(df_sample))]

In [48]:
x_train = torch.tensor(np.array(features), dtype = torch.float).to(device)
y_train = torch.tensor(np.array(df_len.values), dtype = torch.float).to(device)
train_data = list(zip(x_train, y_train))

In [50]:
print(len(train_data))

85533


In [51]:
learning_rate = 1e-5

In [52]:
model = torch.nn.Sequential(
    torch.nn.Linear(11, 32),
    torch.nn.ReLU(),
#     torch.nn.Linear(32, 64),
#     torch.nn.ReLU(),
    torch.nn.Linear(32, 1),
).to(device)

In [53]:
loss_fn = torch.nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [54]:
def train_epoch(train_data, model, optimizer, loss_fn):
    losses = []
    # get a batch of training data
    for x, y in tqdm(train_data):
        # make predictions for this batch
        y_pred = model(x)
        
        # Compute the RSME loss
        loss = torch.sqrt(loss_fn(y_pred, y))
        
        # Backpropagation
        # zero out the gradients so that it will not accumulate through each iteration
        optimizer.zero_grad()
        
        # Compute the gradents with the backward call (backprop)
        loss.backward()
        
        # Update weight using gradient descent 
        optimizer.step()
        
        losses.append(loss.item())
    return np.mean(losses)

In [55]:
total_epochs = 3
train_accs, valid_accs = [], []
max_acc = 0
for epoch in range(total_epochs):
    
    model.train() # gradient tracking is on
    
    train_loss = train_epoch(train_data, model, optimizer, loss_fn)
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss:>0.4f} \n")

100%|██████████| 85533/85533 [01:29<00:00, 955.08it/s] 


Epoch: 1, Train Loss: 397.7182 



100%|██████████| 85533/85533 [01:25<00:00, 998.56it/s] 


Epoch: 2, Train Loss: 351.8845 



100%|██████████| 85533/85533 [01:24<00:00, 1013.42it/s]


Epoch: 3, Train Loss: 348.1718 



 49%|████▊     | 41580/85533 [00:42<00:44, 977.82it/s] 


KeyboardInterrupt: 

In [ ]:
#TODO Implement accuracy or smthing for training
#Do something with validation (make a validation set)
